In [1]:
import os 
os.environ["SPARK_HOME"] = ""
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.2.10 pyspark-shell'

import findspark
findspark.init()
findspark.add_packages(
    ['org.postgresql:postgresql:42.2.10']
)
# findspark.add_jars(
#     [
#         'postgresql-42.6.0.jar'
#     ]
# )


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import col, avg

spark = SparkSession.builder\
    .appName("capstone_project")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/05/26 01:17:54 WARN Utils: Your hostname, pcminh-ThinkPad resolves to a loopback address: 127.0.1.1; using 172.31.163.12 instead (on interface eth0)
23/05/26 01:17:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/pcminh/bigdata-capstone/.venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pcminh/.ivy2/cache
The jars for the packages stored in: /home/pcminh/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-df6905eb-2733-4893-88cc-f0030aea73ec;1.0
	confs: [default]
	found org.postgresql#postgresql;42.2.10 in central
:: resolution report :: resolve 196ms :: artifacts dl 5ms
	:: modules in use:
	org.postgresql#postgresql;42.2.10 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-df6905eb-2733-4893-88cc-f0030aea73ec
	confs:

In [3]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [4]:
from batch.rw import read_table_to_view
read_table_to_view(spark)
spark.sql("SHOW TABLES").show()


23/05/26 01:18:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |     crashes|       true|
|         |redlight_cam|       true|
|         |   speed_cam|       true|
|         |     streets|       true|
|         |traffic_hist|       true|
+---------+------------+-----------+



In [5]:
from batch.dimensions import dim_street, dim_camera_stage, dim_camera_map_street, dim_segment, dim_date, dim_hour
from batch.facts import fact_violation, fact_crash, fact_congestion_batch

In [6]:
d_street = dim_street(spark)

In [7]:
d_street.filter("suffix = ''").show()

23/05/26 01:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:18:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Direction, Street , Suffix, Suffix Direction, Min Address, Max Address
 Schema: direction, street, suffix, suffix_direction, min_address, max_address
Expected: street but found: Street 
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/streets/streets.csv
23/05/26 01:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious pe

+----------+---------+------+------+----------------+----------------+-----------+-----------+
|street_key|direction|street|suffix|suffix_direction|full_street_name|min_address|max_address|
+----------+---------+------+------+----------------+----------------+-----------+-----------+
+----------+---------+------+------+----------------+----------------+-----------+-----------+



In [8]:
d_camera = dim_camera_stage(spark)


In [9]:
d_camera = dim_camera_map_street(spark, d_camera, d_street)

In [10]:
d_street.where('street = "BROADWAY"').show()

23/05/26 01:18:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:18:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:18:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:18:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Direction, Street , Suffix, Suffix Direction, Min Address, Max Address
 Schema: direction, street, suffix, suffix_direction, min_address, max_address
Expected: street but found: Street 
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/streets/streets.csv
23/05/26 01:18:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious pe

+----------+---------+--------+------+----------------+----------------+-----------+-----------+
|street_key|direction|  street|suffix|suffix_direction|full_street_name|min_address|max_address|
+----------+---------+--------+------+----------------+----------------+-----------+-----------+
|       642|        N|BROADWAY|    ST|                |   N BROADWAY ST|       2800|       6399|
+----------+---------+--------+------+----------------+----------------+-----------+-----------+



In [11]:
d_camera.where('street_key IS NULL').collect()

23/05/26 01:18:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:18:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:18:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ADDRESS, CAMERA ID, LATITUDE, LONGITUDE
 Schema: address, camera_id, latitude, longitude
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/speed_cam/speed_cam.csv
23/05/26 01:18:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: INTERSECTION, CAMERA ID, ADDRESS, LATITUDE, LONGITUDE
 Schema: intersection, camera_id, address, latitude, longitude
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/redlight_cam/redlight_cam.csv
23/05/26 01:18

[]

In [14]:
d_camera.show()
d_camera.createOrReplaceTempView('d_camera')

23/05/26 01:19:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:19:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:19:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ADDRESS, CAMERA ID, LATITUDE, LONGITUDE
 Schema: address, camera_id, latitude, longitude
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/speed_cam/speed_cam.csv
23/05/26 01:19:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: INTERSECTION, CAMERA ID, ADDRESS, LATITUDE, LONGITUDE
 Schema: intersection, camera_id, address, latitude, longitude
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/redlight_cam/redlight_cam.csv
23/05/26 01:19

+---------+--------+----------------------+--------------------+----------+------------+-------------+
|camera_id|    type|red_light_intersection|             address|street_key|    latitude|    longitude|
+---------+--------+----------------------+--------------------+----------+------------+-------------+
|     1002|redlight|    WESTERN AND CERMAK|  2200 S WESTERN AVE|      2535| 41.85198384|-87.685785704|
|     1003|redlight|    WESTERN AND CERMAK|    2400 W CERMAK RD|       714|41.852141307|-87.685752642|
|     1011|redlight|  PETERSON AND WESTERN|  6000 N WESTERN AVE|      2533|41.990586042|-87.689822473|
|     1014|redlight|  PETERSON AND WESTERN|     2400 W PETERSON|      2132| 41.99060901|-87.689735481|
|     1023|redlight|  IRVING PARK AND N...|  6400 W IRVING PARK|      1371|41.953024751|-87.786683238|
|     1024|redlight|  IRVING PARK AND N...|  6400 W IRVING PARK|      1371|41.953024751|-87.786683238|
|     1031|redlight|      FOSTER AND NAGLE|        5200 N NAGLE|      196

In [13]:
d_street.show()

23/05/26 01:19:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:19:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:19:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:19:01 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Direction, Street , Suffix, Suffix Direction, Min Address, Max Address
 Schema: direction, street, suffix, suffix_direction, min_address, max_address
Expected: street but found: Street 
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/streets/streets.csv
23/05/26 01:19:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious pe

+----------+---------+------+------+----------------+----------------+-----------+-----------+
|street_key|direction|street|suffix|suffix_direction|full_street_name|min_address|max_address|
+----------+---------+------+------+----------------+----------------+-----------+-----------+
|         1|        E| 100TH|    PL|                |      E 100TH PL|          1|       1199|
|         2|        W| 100TH|    PL|                |      W 100TH PL|        300|       2629|
|         3|        E| 100TH|    ST|                |      E 100TH ST|          1|       4001|
|         4|        W| 100TH|    ST|                |      W 100TH ST|          1|       2799|
|         5|        E| 101ST|    PL|                |      E 101ST PL|          1|        699|
|         6|        W| 101ST|    PL|                |      W 101ST PL|        300|       3999|
|         7|        E| 101ST|    ST|                |      E 101ST ST|          1|       3699|
|         8|        W| 101ST|    ST|              

In [15]:
d_camera.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/d_camera.csv')
d_street.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/d_street.csv')

23/05/26 01:19:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:19:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/26 01:19:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ADDRESS, CAMERA ID, LATITUDE, LONGITUDE
 Schema: address, camera_id, latitude, longitude
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/speed_cam/speed_cam.csv
23/05/26 01:19:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: INTERSECTION, CAMERA ID, ADDRESS, LATITUDE, LONGITUDE
 Schema: intersection, camera_id, address, latitude, longitude
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/redlight_cam/redlight_cam.csv
23/05/26 01:19

In [10]:
d_segment = dim_segment(spark, d_street)

In [12]:
d_segment.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/d_segment.csv')

23/05/24 13:31:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/24 13:31:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/24 13:32:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Full Street Name, Direction, Street , Suffix
 Schema: full_street_name, direction, street, suffix
Expected: full_street_name but found: Full Street Name
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/streets/streets.csv
23/05/24 13:32:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/24 13:32:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [12]:
d_date = dim_date(spark)

In [15]:
d_hour = dim_hour(spark)

In [6]:
f_violation = fact_violation(spark)

In [7]:
f_crash = fact_crash(spark, d_street)

In [20]:
f_congestion = fact_congestion_batch(spark)

In [22]:
f_congestion

DataFrame[record_id: string, segment_id: int, timestamp: timestamp, gmtts_str: string, speed: int, bus_count: int, message_count: int, date_key: int, hour_key: int]

In [8]:
f_crash.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/f_crash.csv')

23/05/25 01:42:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/25 01:42:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/25 01:42:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Full Street Name, Direction, Street , Suffix
 Schema: full_street_name, direction, street, suffix
Expected: full_street_name but found: Full Street Name
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/streets/streets.csv
23/05/25 01:42:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/25 01:42:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/05/25 08:08:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3226770 ms exceeds timeout 120000 ms
23/05/25 08:08:50 WARN SparkContext: Killing executors is not supported by current scheduler.
23/05/25 08:08:50 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.

In [19]:
f_crash.count()

ERROR:root:KeyboardInterrupt while sending command.][Stage 41:>   (0 + 0) / 1]
Traceback (most recent call last):
  File "/home/pcminh/bigdata-capstone/.venv/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/pcminh/bigdata-capstone/.venv/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [16]:
f_crash.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/f_crash.csv')

23/05/24 13:39:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/24 13:39:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/24 13:39:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Full Street Name, Direction, Street , Suffix
 Schema: full_street_name, direction, street, suffix
Expected: full_street_name but found: Full Street Name
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/streets/streets.csv
23/05/24 13:39:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/24 13:39:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
ERROR:root

KeyboardInterrupt: 

In [7]:
f_violation.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/f_violation.csv')

23/05/24 13:28:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CAMERA ID, VIOLATION DATE, VIOLATIONS
 Schema: camera_id, violation_date_mmddyyyy, violations
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/redlight_cam/redlight_cam.csv
23/05/24 13:28:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CAMERA ID, VIOLATION DATE, VIOLATIONS
 Schema: camera_id, violation_date_mmddyyyy, violations
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/speed_cam/speed_cam.csv


In [19]:
d_date.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/d_date.csv')
d_hour.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/d_hour.csv')